# Search Engine Gutenberg

Cells that take some time to run (cells for the indexing and ranking) are commented out in case you decide to run all the cells at once to test the engine. The search query input is the last cell of the notebook.

In [2]:
import numpy as np
import nltk
import copy
import shelve
from nltk.corpus import gutenberg
from collections import Counter
import math
import operator

In [3]:
#global variables
#unwanted = set(nltk.corpus.stopwords.words("english"))
unwanted = set()
unwanted.update(list('!"#$%&\'()*+,-./:;<=>? @[\\]`^_`{|}~£'))
unwanted.update(["--", "'s", "mr", "mrs", "''", "``", "", "'t",])

corpus = gutenberg

n_documents = len(gutenberg.fileids()) #number of documents in the corpus

In [4]:
#method for cleaning the data
def get_rid_of_underscores_and_dots_and_upper(word):
    if word[0]=="_" and word[-1]=="_":
        return word[1:-1].lower()
    elif word[-1]==".":
        return word[:-1].lower()
    elif word[0]=="`":
        return word[1:]
    else:
        return word.lower()

#dividing into tokens - unigrams
def tokenise_and_clean(corpus):
    doc_to_tokens = {}
    for document_id in tqdm(corpus.fileids()):
        tokens = nltk.word_tokenize(corpus.raw(document_id))
        clean_tokens = [get_rid_of_underscores_and_dots_and_upper(t) for t in tokens]
        clean_tokens = filter(lambda x: x not in unwanted, clean_tokens)
        doc_to_tokens[document_id] = list(clean_tokens)
    return doc_to_tokens

#dividing into tokens - bigrams
def bigramise_and_clean(corpus):
    doc_to_bigrams = {}
    for document_id in tqdm(corpus.fileids()):
        words = corpus.words(document_id)
        clean_words = [get_rid_of_underscores_and_dots_and_upper(w) for w in words]
        clean_words = filter(lambda x: x not in unwanted, clean_words)
        doc_to_bigrams[document_id] = list(nltk.bigrams(clean_words))
    return doc_to_bigrams

In [5]:
# method for indexing
from tqdm import tqdm
def index_files(index_filename, documents_to_tokens):
    for document_id in tqdm(documents_to_tokens.keys()):
        frequency_of_all_tokens = Counter(documents_to_tokens[document_id])
        document_len = len(documents_to_tokens[document_id])
        for token in documents_to_tokens[document_id]:
            term_frequency_per_document = (frequency_of_all_tokens[token])/(document_len)
            if token not in index_filename:
                index_filename[token] = []
            if (document_id, term_frequency_per_document) not in index_filename[token]:    
                index_filename[token].append((document_id, term_frequency_per_document))

In [6]:
#takes index in form {'keyword': (document_id, term_frequency)} and n_documents as number of all documents in the corpus
def rank_tf_idf_uni(ranked_index, index, n_document):
    for word in tqdm(index):
        docs_with_word = len(index[word])
        ranked_index[word] = []
        ranked_index.sync()
        for pair in index[word]:
            doc_id = pair[0]
            doc_freq = 1+math.log(n_document/docs_with_word, 2)
            rank = (doc_id, doc_freq*pair[1])
            ranked_index[word].append(rank)
            ranked_index.sync()

In [7]:
def rank_tf_idf_bi(ranked_index, index, n_document):
    for bigram in tqdm(index):
        docs_with_word = len(index[bigram])
        word = " ".join(list(bigram))
        ranked_index[word] = []
        ranked_index.sync()
        for pair in index[bigram]:
            doc_id = pair[0]
            doc_freq = 1+math.log(n_document/docs_with_word, 2)
            rank = (doc_id, doc_freq*pair[1])
            ranked_index[word].append(rank)
            ranked_index.sync()

In [8]:
#takes query_tfidfs as {'keyword': tfidf} and doc_per_word as {'keyword': tfidf} in one document
def cosine_similarity(query_tfidfs, tfidf_per_word):
    query_sum = 0
    doc_sum = 0
    dot_product = 0
    for query_word in query_tfidfs:
        query_sum += (query_tfidfs[query_word]**2)
        doc_sum += (tfidf_per_word[query_word]**2)
        dot_product += (query_tfidfs[query_word] * tfidf_per_word[query_word])
    query_norm = math.sqrt(query_sum)
    doc_norm = math.sqrt(doc_sum)
    if query_norm * doc_norm == 0:
        return 0
    else:
        return dot_product/(query_norm * doc_norm)

In [9]:
def get_tfidf_for_document_id(document_id, subset_from_index):
    tfidfs_per_word = {}
    for word in subset_from_index:
        for pair in subset_from_index[word]:
            if pair[0] == document_id:
                tfidfs_per_word[word] = pair[1]
        #in case some query words are not present in the document
        if word not in tfidfs_per_word:
            tfidfs_per_word[word] = 0
    
    return tfidfs_per_word

In [10]:
# takes result as a tuple of unigram_index search and bigram_index search, both as [(document_id, rank), ..]
# prints example sentences for each document where query terms are present
def find_example(results, query_unigrams, query_bigrams, corpus):
    substring_window = 20 #to show the query with this many symbols on both sides
    top_uni = results[0]
    top_bi = results[1]
    result_counter = 0 #number of results already shown
    printed_docs = [] #to store docs we've already shown
    
    for pair in top_bi:
        if pair[1] > 0 and result_counter < 8:
            print("\033[1m"+pair[0].upper()+"\033[0m")
            result_counter += 1
            printed_docs.append(pair[0])
            for bigram in query_bigrams:
                try:
                    bg = " ".join(list(bigram))
                    occurence_index = corpus.raw(pair[0]).lower().index(bg)
                    if occurence_index < substring_window:
                        print(corpus.raw(pair[0])[:occurence_index+substring_window]+"...\n")
                    else:
                        print("..."+corpus.raw(pair[0])[occurence_index-substring_window:occurence_index+substring_window]+"...\n")
                except ValueError:
                    continue
    
    #if there aren't enough results, we can go to the unigram index and supply more documents with parts of the query
    # '4' is an arbitrary number, we continue until we've got 8 documents
    if result_counter < 4:
        for pair in top_uni:
            if pair[1] > 0 and pair[0] not in printed_docs and result_counter < 8:
                print("\033[1m"+pair[0].upper()+"\033[0m")
                result_counter += 1
                printed_docs.append(pair[0])
                for unigram in query_unigrams:
                    try:
                        occurence_index = corpus.raw(pair[0]).lower().index(unigram)
                        if occurence_index < substring_window:
                            print(corpus.raw(pair[0])[:occurence_index+substring_window]+"...\n")
                        else:
                            print("..."+corpus.raw(pair[0])[occurence_index-substring_window:occurence_index+substring_window]+"...\n")
                    except ValueError:
                        pass
            

In [11]:
# method for searching: matching query and corpus documents
def search(ranked, ranked_bigrams, query, corpus):
    #dividing query into unigrams and bigrams
    q_tokens = nltk.word_tokenize(query)
    clean_q_tokens = [get_rid_of_underscores_and_dots_and_upper(t) for t in q_tokens]
    bg_clean_q_tokens = clean_q_tokens
    
    clean_q_tokens = filter(lambda x: x not in unwanted, clean_q_tokens)
    bg_clean_q_tokens = filter(lambda x: x not in unwanted, bg_clean_q_tokens)
    
    query_bigrams = list(nltk.bigrams(bg_clean_q_tokens))
    
    list_of_tokens = list(clean_q_tokens)
    query_length = len(list_of_tokens)
    
    query_bg_length = len(query_bigrams)
    
    #calculating tf-idfs for query terms in unigrams
    frequencies = Counter(list_of_tokens)
    frequency_of_query_terms = {}
    ranked_docs_per_term = {}
    for term in frequencies:
        if term in ranked:
            frequency_of_query_terms[term] = frequencies[term]/query_length
            term_idf = 1+math.log(n_documents/len(ranked[term]), 2)
            frequency_of_query_terms[term] *= term_idf
            ranked_docs_per_term[term] = ranked[term]
            
    #calculating tf-idfs for query bigrams
    bg_frequencies = Counter(query_bigrams)
    bg_frequency_of_query_terms = {}
    bg_ranked_docs_per_term = {}
    for pair in bg_frequencies:
        term = " ".join(list(pair))
        if term in ranked_bigrams:
            bg_frequency_of_query_terms[term] = bg_frequencies[pair]/query_bg_length
            term_idf = 1+math.log(n_documents/len(ranked_bigrams[term]), 2)
            bg_frequency_of_query_terms[term] *= term_idf
            bg_ranked_docs_per_term[term] = ranked_bigrams[term]
    
    #if no query words exist in our indexes
    if len(ranked_docs_per_term) == 0 and len(bg_ranked_docs_per_term) == 0:
        print("Please try another key phrase, we can't seem to find this one, sorry...\n")
        return
    
    #comparing similarities of query and docs vectors
    ranked_docs_per_query = {}
    bg_ranked_docs_per_query = {}
    for document_id in corpus.fileids():
        tfidfs_per_word = get_tfidf_for_document_id(document_id, ranked_docs_per_term)
        bg_tfidfs_per_word = get_tfidf_for_document_id(document_id, bg_ranked_docs_per_term)
        cs = cosine_similarity(frequency_of_query_terms, tfidfs_per_word)
        bg_cs = cosine_similarity(bg_frequency_of_query_terms, bg_tfidfs_per_word)
        if cs != 0 or bg_cs != 0:
            ranked_docs_per_query[document_id] = cs
            bg_ranked_docs_per_query[document_id] = bg_cs
    
    sorted_ranked_docs = sorted(ranked_docs_per_query.items(), key=operator.itemgetter(1), reverse=True)
    bg_sorted_ranked_docs = sorted(bg_ranked_docs_per_query.items(), key=operator.itemgetter(1), reverse=True)
    results = (sorted_ranked_docs, bg_sorted_ranked_docs)
    find_example(results, list_of_tokens, query_bigrams, corpus)


In [11]:
'''# a cell that was used to prepare the data

# preparing data
print("I'm tokenising")
docs_to_tokens = tokenise_and_clean(corpus)
docs_to_bigrams = bigramise_and_clean(corpus)

# creating two indexes: unigram and bigram
print("I'm indexing")
index_dictionary = {}
index_bigrams = {}
index_files(index_dictionary, docs_to_tokens)
index_files(index_bigrams, docs_to_bigrams)'''

  0%|          | 0/18 [00:00<?, ?it/s]

I'm tokenising


  0%|          | 0/18 [00:00<?, ?it/s]

I'm indexing


100%|██████████| 18/18 [00:08<00:00,  2.61it/s]


In [12]:
'''# calculating TF-IDF for our two indexes
print("I'm ranking")
with shelve.open('eng_index_uni',writeback=True) as ranked:
    ranked.sync()
with shelve.open('eng_index_bi',writeback=True) as ranked_bigrams:
    ranked_bigrams.sync()
ranked = shelve.open("eng_index_uni", writeback=True)
rank_tf_idf_uni(ranked, index_dictionary, n_documents)
ranked_bigrams = shelve.open("eng_index_bi", writeback=True)
rank_tf_idf_bi(ranked_bigrams, index_bigrams, n_documents)
print("Done!")
ranked.close()
ranked_bigrams.close()'''

  0%|          | 142/52086 [00:00<00:36, 1408.17it/s]

I'm ranking


100%|██████████| 608635/608635 [00:30<00:00, 20157.30it/s]


Done!


In [12]:
# actual searching
ranked = shelve.open("eng_index_uni", writeback=True)
ranked_bigrams = shelve.open("eng_index_bi", writeback=True)
query = input("What would you like to search for?\n")        
search(ranked, ranked_bigrams, query, corpus)

What would you like to search for?
Moby Dick is a great novel by a great artist
MELVILLE-MOBY_DICK.TXT
[Moby Dick by Herman ...

.... and Dan. HVAL.  This animal is named f...

...e Lord had prepared a great fish to swal...

...OLOGY.

(Supplied by a Late Consumptiv...

...e Lord had prepared a great fish to swal...

BRYANT-STORIES.TXT
...ed and indeed there is another lion! And...

...


Once there was a great big jungle; ...

...ttle Tulip said.

By and by she heard ...

...


Once there was a great big jungle; ...

AUSTEN-PERSUASION.TXT
... one fair claim on his attachment;
since...

...ension, entitled to a great deal
of comp...

...ter had improved it by adding, for the i...

...ension, entitled to a great deal
of comp...

EDGEWORTH-PARENTS.TXT
...ssmore, in Ireland, is a small cabin,
i...

...by which she earned a great deal
more t...

...ce, all was managed by a Mr.
Hopkins, a...

...by which she earned a great deal
more t...

AUSTEN-SENSE.TXT
...omfortably spent.  His attachmen